In [231]:
import folium
import re
import fileinput
import pandas as pd
from folium import IFrame
from folium import plugins

In [232]:
VEGAS_COORDINATES = (36.1699, -115.1398)
map = folium.Map(location=VEGAS_COORDINATES, zoom_start=11)

In [233]:
pos_df = pd.read_csv('pos_bsns.csv')
neg_df = pd.read_csv('neg_bsns.csv')
del(pos_df['Unnamed: 0'])
del(neg_df['Unnamed: 0'])

In [234]:
pos_topics = {0:'menu/experience', 1:'burger/fries', 2:'service/atmosphere', 
              3:'breakfast/pancakes', 4:'chicken/waffles', 5:'location/atmosphere'}

neg_topics = {0:'wait/table', 1:'burger/fries', 2:'bad_service', 
              3:'chicken/salad', 4:'breakfast/eggs', 5:'buffet/price'}

In [235]:
def get_pos_topics(id):
    positive = pos_df.loc[pos_df['business_id'] == "-CQokjildrY7UZezXCdEBw"]
    pos_scores = [positive['0'].get(0), positive['1'].get(0), positive['2'].get(0), 
                  positive['3'].get(0), positive['4'].get(0), positive['5'].get(0)]
    return pos_scores

def html_text(business_id, stars, neg_scores, name):
    html_topics1 = ['','','','','','']
    html_topics2 = ['','','','','','']
    
    pos_scores = get_pos_topics(business_id)
    for m in pos_topics:
        html_topics1[m] = 'Topic {}: {:18} {:.2f}%'.format(m, pos_topics[m], pos_scores[m]*100)
    for n in neg_topics:
        html_topics2[n] = 'Topic {}: {:18} {:.2f}%'.format(n, neg_topics[n], neg_scores[n]*100)
        
    pos_topics_text = 'Positive Topics<br>' + '<br>'.join(html_topics1)
    neg_topics_text = 'Negative Topics<br>' + '<br>'.join(html_topics2)
    stars_text = '<h3> Stars: ' + str(stars) + '</h3>'
    name_text = '<h3> Name: ' + name + '</h3>'
    return name_text + stars_text + '<p>' + pos_topics_text + '<br><br>' + neg_topics_text + '</p>'

def get_popup(business_id, stars, neg_scores, name):
    html = "" + html_text(business_id, stars, neg_scores, name) + ""
    iframe = folium.IFrame(html=html, width=300, height=200)
    return folium.Popup(iframe, max_width=1000)

In [236]:
def star_color(stars):
    if stars <= 1.5:
        return '#FF0000'
    elif stars <= 2.5:
        return '#FFAD00'
    elif stars <= 3.5:
        return '#FFFF00'
    elif stars < 4.5:
        return '#A1FF00'
    else:
        return '#007200'
    
def get_icon(stars):
    return plugins.BeautifyIcon(border_color='transparent', background_color=star_color(stars), 
                                icon='circle', icon_shape='marker', text_color='#FFF')
    

In [237]:
for row in neg_df.itertuples():
    #print(row)
    probs = [row._7, row._8, row._9, row._10, row._11, row._12]
    folium.Marker(location=[row.latitude, row.longitude], popup=get_popup(row.business_id, row.stars_y, probs, row.name), 
                  icon=get_icon(row.stars_y)).add_to(map)
    if row.Index > 600:
        break

In [238]:
import branca.colormap as cm
colormap = cm.LinearColormap(
    ['red', 'orange', 'yellow', 'green'],
    vmin=0, vmax=5
)
colormap.caption = 'Star Rating'
map.add_child(colormap)

In [239]:
display(map)

In [240]:
map.save('vegas.html')

In [241]:
with open("vegas.html") as inf:
    txt = inf.read()

#Find all the markers names given by folium
markers = re.findall(r'\bmarker_\w+', txt)
markers = list(set(markers))

for linenum,line in enumerate( fileinput.FileInput("vegas.html",inplace=1) ):
   pattern = markers[0] + ".bindPopup"
   pattern2 = markers[0] + ".on('mouseover', function (e) {this.openPopup();});"
   pattern3 = markers[0] + ".on('mouseout', function (e) {this.closePopup();});"

   if pattern in line:
      print(line.rstrip())
      print(pattern2)
      print(pattern3)
   else:
      print(line.rstrip())